# Using preprocessed files either english or german  for automatic translation from english to german using T5 from german to Polish MarianMTModel

# 1. Installing required libraries 

In [ ]:
# Install transformers library.
!pip install -q git+https://github.com/huggingface/transformers.git
# Install helper functions.
!pip install -q git+https://github.com/gmihaila/ml_things.git
!pip install sentencepiece #required by Marian

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 71kB 4.8MB/s 
     |████████████████████████████████| 1.2MB 10.8MB/s 


In [ ]:
from transformers import MarianMTModel, MarianTokenizer
from transformers import AutoModelWithLMHead
from transformers import pipeline
import torch

from IPython.display import HTML
from IPython.display import Image

from tqdm import tqdm

# 2. Mounting google drive

In [ ]:
#Mount Google Drive as folder
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# 3. Initiating auxilary funcions

In [ ]:
model = AutoModelWithLMHead.from_pretrained("t5-base") #loading t5 model in translation mode eng->de
translator = pipeline("translation_en_to_de")

model_name = 'Helsinki-NLP/opus-mt-de-pl'  #loading marian  model translator de->pl
tokenizer = MarianTokenizer.from_pretrained(model_name)
model_m = MarianMTModel.from_pretrained(model_name)

/usr/local/lib/python3.6/dist-packages/transformers/models/auto/modeling_auto.py:958: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [ ]:
def temp_list(text_out):
  text_all=[]
  next_line=''
  dot_line='...................................'
  text_all.append(next_line)
  text_all.append(dot_line)
  for t in text_out:
    text_all.append(t)
  return text_all

temp_de =[]
file_name= '/content/drive/MyDrive/GPT/en_de_pl_trans.txt'
def save_list_to_file(file_name, temp_lista=temp_de):
    with open(file_name, 'a') as f:
        for item in temp_lista:
            f.write("%s\n" % item)

def translate_ge_to_pl(src_text):
    translated = model_m.generate(**tokenizer.prepare_seq2seq_batch(src_text, return_tensors="pt"))
    tgt_text = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
    
#     print(tgt_text)
    return tgt_text

def trans_en_de(text_out):
  temp=[]
  for text in text_out:
    text_out_de= translator(text)

    src_text= text_out_de[0]['translation_text']
    temp.append(src_text)
  return temp


# 4. Translation form

In [ ]:
#@title 2.Translating preprocessed( new line \n every sentence) txt files to german and polish { display-mode: "form" }

en_text_file = "/content/drive/MyDrive/trans_pl/text_eng_relined.txt" #@param [""] {allow-input: true}

ger_text_file = "" #@param [""] {allow-input: true}
text_file= file_name


if len(en_text_file) >0:
  output_de= en_text_file[:-4]+'_de.txt'
  output_pl= en_text_file[:-4]+'_pl.txt'

  print('Text file {} will be translated first to {}'.format(en_text_file,output_de))
  print(' ...  and then to polish {}'.format(output_pl))
    
  text_out=[]
  with open(en_text_file) as file :
    for line in file :
      line.strip()
      text_out.append(line)
    
    
  
  print('\n 1. ==== > Translating to german ...')
  lines=[]
  for text in tqdm(text_out):
    text_temp= translator(text)
    src_text= text_temp[0]['translation_text']
    lines.append(src_text)
  #writing list to file attaching after every list item end of line sign \n
  save_list_to_file(output_de, temp_lista=lines)
  print('\n =====> File  {} was saved'.format(output_de))

  print( '\n 2. ==== > Translating to polish ...')
  lines_pl =[]
  for l in tqdm(lines):
    
    text_out_pl= translate_ge_to_pl(l)
    for t in text_out_pl:
      lines_pl.append(t)

  #writing list to file attaching after every list item end of line sign \n
  save_list_to_file(output_pl, temp_lista=lines_pl) 
  print('\n 3, =====> File  {} was saved'.format(output_pl))



else:
  if len(ger_text_file) >0:
    output_pl= ger_text_file[:-4]+'_pl.txt'
    print('Text file {} will be translated  to {} '.format(ger_text_file,output_pl))

    lines=[]
    with open(ger_text_file) as file :
      for line in file :
        line.strip()
        lines.append(line)
    
    print( '\n 1. ==== > Translating to polish ...')
    lines_pl =[]
    for l in tqdm(lines):
      
      text_out_pl= translate_ge_to_pl(l)
      for t in text_out_pl:
        lines_pl.append(t)

    #writing list to file attaching after every list item end of line sign \n
    save_list_to_file(output_pl, temp_lista=lines_pl)
    print('\n =====> File  {} was saved'.format(output_pl))

  


  else :
    print( 'You need to give at least one input text file')

  0%|          | 0/25 [00:00<?, ?it/s]

Text file /content/drive/MyDrive/trans_pl/text_eng_relined.txt will be translated first to /content/drive/MyDrive/trans_pl/text_eng_relined_de.txt
 ...  and then to polish /content/drive/MyDrive/trans_pl/text_eng_relined_pl.txt

 1. ==== > Translating to german ...


  0%|          | 0/25 [00:00<?, ?it/s]


 =====> File  /content/drive/MyDrive/trans_pl/text_eng_relined_de.txt was saved

 2. ==== > Translating to polish ...


100%|██████████| 25/25 [01:25<00:00,  3.42s/it]


 3, =====> File  /content/drive/MyDrive/trans_pl/text_eng_relined_pl.txt was saved
